# Any and all pacakges used for cleaning and modeling

In [1]:
import pandas as pd
# !pip install wrds
import wrds
import os
import zipfile

c:\Users\ziggy\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Part 1: Merging SPAC and IPO data
- Key Points: Self cleaned excel files with offering dates and firm names
- Both read in seperately and merged on the date variabe

In [2]:
spac_data = pd.read_csv("inputs/SPACs2016-2021.csv", header=0)
IPO_data = pd.read_csv("inputs/IPO_data.csv", header=0)


In [3]:
IPO_data['CUSIP'] = IPO_data['CUSIP'].apply(lambda x: str(x).zfill(9))
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15443,15443,12/15/2023,Bayview Acquisition,000000nan
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104
15445,15445,12/19/2023,Linkage Global Inc,G5500B102


In [4]:
IPO_data = IPO_data[IPO_data['CUSIP'] != '000000nan']
IPO_data

,Unnamed: 0,offer date,IPO name,CUSIP
0,0,01/30/1975,ROYSTER,078088610
1,1,06/09/1975,VARCO,092212610
2,2,06/10/1975,COORS ADOLPH,021701610
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410
4,4,08/26/1975,C. F. BRAUN,010564710
...,...,...,...,...
15440,15440,12/01/2023,Garden Stage Ltd,G3730L107
15441,15441,12/04/2023,Aimei Health Technology Co.,G01341117
15442,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109
15444,15444,12/15/2023,ZKH GROUP LTD,98877R104


In [5]:
spac_data['CUSIP'] = spac_data['CUSIP'].apply(lambda x: str(x).zfill(9))
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1009,CF Corp.,5/25/2016,000000nan
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201
1012,Silver Run Acquisition Corp.,2/29/2016,82811P200


In [6]:
spac_data = spac_data[spac_data['CUSIP'] != '000000nan']
spac_data

,SPAC IPO,Date of IPO,CUSIP
0,Aetherium Acquisition Corp.,12/29/2021,00809J200
1,Welsbach Technology Metals Acquisition Corp.,12/27/2021,950415208
2,Arogo Capital Acquisition Corp.,12/23/2021,042644203
3,Gardiner Healthcare Acquisitions Corp.,12/21/2021,365506203
4,Larkspur Health Acquisition Corp.,12/20/2021,51724W206
...,...,...,...
1007,M III Acquisition Corp.,7/7/2016,55378T203
1008,"Landcadia Holdings, Inc.",6/1/2016,51476W206
1010,KLR Energy Acquisition Corp.,3/16/2016,777385204
1011,Jensyn Acquisition Corp.,3/7/2016,47632B201


In [7]:
merged_IPOnSPAC = pd.merge(left=IPO_data, right=spac_data, how='left', on='CUSIP')
merged_IPOnSPAC['SPAC IPO '] = merged_IPOnSPAC['SPAC IPO '].notna().astype(int)
merged_IPOnSPAC = merged_IPOnSPAC.drop(merged_IPOnSPAC.columns[-1], axis=1)
merged_IPOnSPAC.to_csv('inputs/MergedIPO&SPAC.csv')



In [8]:
merged_IPOnSPAC

,Unnamed: 0,offer date,IPO name,CUSIP,SPAC IPO
0,0,01/30/1975,ROYSTER,078088610,0
1,1,06/09/1975,VARCO,092212610,0
2,2,06/10/1975,COORS ADOLPH,021701610,0
3,3,07/15/1975,KEYSTONE FOODS CORP,049348410,0
4,4,08/26/1975,C. F. BRAUN,010564710,0
...,...,...,...,...,...
15318,15440,12/01/2023,Garden Stage Ltd,G3730L107,0
15319,15441,12/04/2023,Aimei Health Technology Co.,G01341117,0
15320,15442,12/14/2023,INNO HOLDINGS INC.,4576JP109,0
15321,15444,12/15/2023,ZKH GROUP LTD,98877R104,0


In [9]:
# # Convert 'offer date' from 'YYYYMMDD' to 'MM/DD/YYYY'
# IPO_data['offer date'] = pd.to_datetime(IPO_data['offer date'],format='%Y%m%d')
# IPO_data['offer date'] = IPO_data['offer date'].dt.strftime('%m/%d/%Y')

# IPO_data.to_csv("inputs/IPO_data.csv")



In [10]:
# IPO_data.rename(columns={'offer date' : 'Date of IPO'}, inplace=True)

In [11]:
#Reading in ccm data

ccm_data = pd.read_csv('inputs/cleaned_ccm.csv')
ccm_data

,Unnamed: 0,cusip,scf,src,apdedate,fdate,pdate,acchg,acominc,acox,...,re,revt,sale,seq,teq,tstk,txt,wcap,naicsh,mkvalt
0,0,74406A102,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2035.297,5883.549,5883.549,2032.183,NaN,5.976,434.460,NaN,522110.0,16439.8825
1,1,74406A102,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1981.166,5529.908,5529.908,1907.511,NaN,76.517,92.330,NaN,522110.0,1011.1323
2,2,74406A102,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2212.848,4072.630,4072.630,2139.072,NaN,76.685,49.006,NaN,522110.0,1878.1411
3,3,74406A102,NaN,5.0,2003-12-31,NaN,NaN,NaN,NaN,NaN,...,2337.966,2781.408,2781.408,2325.449,NaN,15.425,128.088,NaN,522110.0,3380.8031
4,4,68234X102,7.0,5.0,2013-12-31,2014-04-14,NaN,0.0,0.014,2.527,...,-174.412,37.779,37.779,118.122,118.122,0.000,1.944,256.727,325414.0,867.8290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27783,27783,04522R200,7.0,5.0,2018-12-31,2019-06-26,2019-03-23,0.0,0.000,0.000,...,-132.469,0.000,0.000,30.618,30.618,0.000,0.015,21.095,325414.0,NaN
27784,27784,G6700G107,7.0,5.0,2018-12-31,2019-03-01,2019-01-31,0.0,-107.800,12.300,...,-24.400,2213.600,2213.600,2687.100,2687.100,0.000,37.900,384.800,3323.0,3980.4510
27785,27785,Y28895103,7.0,5.0,2018-12-31,2019-06-14,NaN,0.0,-4.150,7.595,...,-11.190,319.018,319.018,292.503,292.503,0.000,1.389,66.283,483111.0,118.9594
27786,27786,Y62151108,7.0,5.0,2018-12-31,2019-04-08,2019-03-10,0.0,0.000,2.903,...,NaN,133.921,133.921,182.473,182.473,0.000,0.000,-22.247,483111.0,97.9265


In [12]:

merged_IPOnSPAC.rename(columns={'CUSIP' : 'cusip'}, inplace=True)
merge_df_ccm = pd.merge(merged_IPOnSPAC, ccm_data, on='cusip', how='inner')

merge_df_ccm

,Unnamed: 0_x,offer date,IPO name,cusip,SPAC IPO,Unnamed: 0_y,scf,src,apdedate,fdate,...,re,revt,sale,seq,teq,tstk,txt,wcap,naicsh,mkvalt
0,2089,06/05/1986,Genzyme,372917104,0,90,7.0,5.0,NaN,NaN,...,480.941,752.483,752.483,1750.280,NaN,0.901,57.140,438.733,325414.0,8587.5923
1,2089,06/05/1986,Genzyme,372917104,0,91,7.0,5.0,NaN,NaN,...,530.024,981.926,981.926,2280.352,NaN,0.901,16.169,478.191,325414.0,12754.5498
2,2089,06/05/1986,Genzyme,372917104,0,92,7.0,5.0,NaN,NaN,...,772.773,1080.185,1080.185,2585.884,NaN,0.901,28.721,825.573,325414.0,6348.8860
3,4619,06/16/1992,Columbia Banking System,197236102,0,7951,NaN,53.0,NaN,NaN,...,34.181,67.914,67.914,78.800,NaN,0.000,6.122,NaN,522110.0,122.3405
4,4619,06/16/1992,Columbia Banking System,197236102,0,7952,NaN,5.0,NaN,NaN,...,43.033,111.436,111.436,135.301,NaN,0.000,6.406,NaN,522110.0,269.3655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10499,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,21344,7.0,5.0,2006-12-31,2007-05-09,...,44.031,15.967,15.967,96.751,NaN,0.000,1.046,NaN,522120.0,162.5353
10500,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,21345,7.0,5.0,2007-12-31,2008-05-30,...,55.807,18.407,18.407,108.829,NaN,4.665,9.150,NaN,522120.0,150.3238
10501,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,21346,7.0,5.0,2008-12-31,2009-04-13,...,57.217,23.741,23.741,110.502,NaN,4.407,1.178,NaN,522120.0,88.7210
10502,14676,07/13/2021,"NorthEast Community Bancorp, I",664121100,0,21347,7.0,5.0,2009-12-31,2010-04-01,...,53.967,25.871,25.871,107.448,107.448,4.147,-2.812,NaN,522120.0,84.1617
